In [18]:
import pandas as pd
import os
import json
import requests
import json
import matplotlib.pyplot as plt

DATADIR = "../../dataset"

def get_name_id_dict (wiki):
    '''
    takes the wiki dataset and build a taxon_name to id code lookup dict 
    return a dict, where keys are taxon names, values are lists of corresponding id (Q code) 
    '''
    name_id_dict = {}
    for idx in wiki.index:
        name = wiki.loc[idx,"taxon_name"]
        if name not in name_id_dict:
            name_id_dict[name] = [idx]
        else:
            name_id_dict[name].append(idx)
    return name_id_dict

def link_data(pbdb,name_id_lookup):
    '''
    link the pbdb data with wiki data by appending to pbdb 1, a column "wiki_ref"
    to the pbdb
    where for each entry it contains a list of wiki Q code(s) that 
    corresponds to the fossil record, if there is no match then the list
    is empty, 2, the 
    the matching priority is:
    accepted_name -> genus -> family -> order -> class -> phylum
    '''
    matching_priorities = ["accepted_name", "genus","family","order","class","phylum"]
    wiki_ref_list, wiki_ref_rank_list, lowest_matched_rank = [], [], []
    #pbdb["wiki_ref"], pbdb["wiki_ref_rank"], pbdb["lowest_matched_rank"]=[np.nan]*pbdb.shape[0],[np.nan]*pbdb.shape[0],[np.nan]*pbdb.shape[0]
    for i, row in pbdb.iterrows():
        if i % 10000 == 0:
            print(i)
        for testing_rank in matching_priorities:
            name = row[testing_rank]
            if name in name_id_lookup:
                codes = name_id_lookup[name]
                ranks = [wiki.loc[code]["taxon_rank"] if str(wiki.loc[code]["taxon_rank"]) != "nan" else "nan" for code in codes]
                wiki_ref_list.append("_".join(codes)) 
                wiki_ref_rank_list.append("_".join(ranks)) 
                if testing_rank == "accepted_name":
                    lowest_matched_rank.append(row["accepted_rank"]) 
                else:
                    lowest_matched_rank.append(testing_rank) 
                break
        if len(wiki_ref_list) == i:
            wiki_ref_list.append(np.nan) 
            wiki_ref_rank_list.append(np.nan)
            lowest_matched_rank.append(np.nan) 
    pbdb["wiki_ref"], pbdb["wiki_ref_rank"], pbdb["lowest_matched_rank"] = wiki_ref_list, wiki_ref_rank_list, lowest_matched_rank
    return pbdb

In [17]:
import numpy as np
str(np.nan)

'nan'

In [2]:
# load wikidata and pbdb data
wiki, pbdb = pd.read_csv(os.path.join(DATADIR, "wikidata","processed","data.csv"), index_col = "id"), \
pd.read_csv(os.path.join(DATADIR, "pbdb","data.csv"))

In [10]:
name_id_lookup = get_name_id_dict(wiki)

In [19]:
pbdb = link_data(pbdb,name_id_lookup)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
1060000
1070000
1080000
1090000
1100000
1110000
1120000
1130000
1140000
1150000
1160000
1170000
1180000
1190000
1200000
1210000
1220000
1230000
1240000
1250000
1260000
1270000
1280000
1290000
1300000
1310000
1320000
1330000
1340000


In [21]:
pbdb.shape

(1346120, 21)

In [22]:
pbdb[pbdb["wiki_ref"].isna()]

,occurrence_no,identified_name,accepted_name,identified_rank,accepted_rank,phylum,class,order,family,genus,...,min_ma,lng,lat,paleomodel,paleolng,paleolat,geoplate,wiki_ref,wiki_ref_rank,lowest_matched_rank
49,72817,Buthotrephis gracilis,Buthotrephis gracilis,species,species,NaN,NaN,NaN,NaN,Bythotrephis,...,443.7,-84.000000,39.000000,gp_mid,-111.97,-25.52,101,NaN,NaN,NaN
75,74320,Buthotrephis cf. flexuosa,Bythotrephis,species,genus,NaN,NaN,NaN,NaN,Bythotrephis,...,449.5,-78.634445,38.507221,gp_mid,-110.82,-24.59,101,NaN,NaN,NaN
76,74321,Buthotrephis sp.,Bythotrephis,genus,genus,NaN,NaN,NaN,NaN,Bythotrephis,...,449.5,-78.634445,38.507221,gp_mid,-110.82,-24.59,101,NaN,NaN,NaN
82,77939,Asphaltina sp.,Asphaltina,genus,genus,NaN,NaN,NaN,NaN,Asphaltina,...,314.6,-107.301666,41.805832,gp_mid,-39.19,-1.81,101,NaN,NaN,NaN
83,78002,Asphaltina sp.,Asphaltina,genus,genus,NaN,NaN,NaN,NaN,Asphaltina,...,314.6,-107.301666,41.805832,gp_mid,-39.19,-1.81,101,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1315775,1523176,n. gen. Dianchicystis n. sp. jianshanensis,Dianchicystis jianshanensis,species,species,NaN,NaN,NaN,Vetulocystidae,Dianchicystis,...,513.0,102.599998,24.785000,gp_mid,66.73,12.43,611,NaN,NaN,NaN
1326097,1536154,Convolutubus dargazinensis,Convolutubus dargazinensis,species,species,NaN,NaN,NaN,NaN,Convolutubus,...,541.0,55.833332,31.700001,gp_mid,NaN,NaN,coordinates not computable using this model,NaN,NaN,NaN
1328110,1539781,Limacites n. sp. liasokeuperinus,Limacites liasokeuperinus,species,species,NaN,NaN,NaN,NaN,Limacites,...,199.3,11.392000,50.118999,gp_mid,NaN,NaN,coordinates not computable using this model,NaN,NaN,NaN
1329894,1541959,Dictyorhabdus priscus,Dictyorhabdus priscus,species,species,NaN,NaN,NaN,NaN,Dictyorhabdus,...,449.5,-105.221947,38.470833,gp_mid,NaN,NaN,coordinates not computable using this model,NaN,NaN,NaN


In [23]:
pbdb[pbdb["wiki_ref"].str.contains("_", na= False)]

,occurrence_no,identified_name,accepted_name,identified_rank,accepted_rank,phylum,class,order,family,genus,...,min_ma,lng,lat,paleomodel,paleolng,paleolat,geoplate,wiki_ref,wiki_ref_rank,lowest_matched_rank
45,45807,Caryophyllidae ? indet.,Caryophyllidae,subclass,subclass,Spermatophyta,Magnoliopsida,NaN,NaN,NaN,...,66.00,-176.000000,-44.333332,gp_mid,-126.78,-54.98,807,Q1765819_Q18197550,subclass_family,subclass
192,128542,Cooksonia sp.,Cooksonia,genus,genus,Psilophytophyta,Psilophytopsida,Psilophytales,NaN,Cooksonia,...,427.40,-7.000000,52.000000,gp_mid,-56.14,-17.22,315,Q131986_Q1764467,genus_genus,genus
228,128799,cf. Cooksonia caledonica,Cooksonia,species,genus,Psilophytophyta,Psilophytopsida,Psilophytales,NaN,Cooksonia,...,423.00,-3.000000,51.000000,gp_mid,-57.00,-22.63,315,Q131986_Q1764467,genus_genus,genus
236,128808,Cooksonia sp.,Cooksonia,genus,genus,Psilophytophyta,Psilophytopsida,Psilophytales,NaN,Cooksonia,...,419.20,-3.000000,51.000000,gp_mid,-46.97,-10.16,315,Q131986_Q1764467,genus_genus,genus
240,128812,cf. Cooksonia cambrensis,Cooksonia,species,genus,Psilophytophyta,Psilophytopsida,Psilophytales,NaN,Cooksonia,...,427.40,-7.000000,52.000000,gp_mid,-56.14,-17.22,315,Q131986_Q1764467,genus_genus,genus
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1346057,1564107,Epicheloniceras cf. claudii,Epicheloniceras claudii,species,species,Mollusca,Cephalopoda,Ammonitida,Douvilleiceratidae,Epicheloniceras,...,112.03,138.131500,35.791500,gp_mid,NaN,NaN,coordinates not computable using this model,Q23753267_Q12256927,subgenus_genus,genus
1346068,1564118,Helius n. sp. turolensis,Helius turolensis,species,species,Arthropoda,Insecta,Diptera,Limoniidae,Helius,...,99.60,-0.815000,40.799999,gp_mid,16.53,36.89,304,Q21438473_Q14565944,subgenus_genus,genus
1346078,1564128,Tetragramma sp.,Tetragramma,genus,genus,Echinodermata,Echinoidea,NaN,Pseudodiadematidae,Tetragramma,...,112.03,33.483002,30.792999,gp_mid,NaN,NaN,coordinates not computable using this model,Q25366022_Q23005736,genus_genus,genus
1346080,1564130,"""Cidaris"" sinaiensis",Cidaris sinaiensis,species,species,Echinodermata,Echinoidea,Cidaroida,Cidaridae,Cidaris,...,112.03,33.483002,30.792999,gp_mid,NaN,NaN,coordinates not computable using this model,Q21444390_Q785144,genus_genus,genus


In [ ]:
pbdb[(pbdb["wiki_ref"].notna()) & (~pbdb["wiki_ref"].str.contains("_", na= False))]

In [25]:
included_pbdb = pbdb[(pbdb["wiki_ref"].notna()) & (~pbdb["wiki_ref"].str.contains("_", na= False))].rename(columns={'occurrence_no': 'id', "accepted_name":"name","accepted_rank":"rank","min_ma":"minma","max_ma":"maxma","wiki_ref":"wikiRef"})[["id","wikiRef","name","rank","minma","maxma","lng","lat"]].to_json(orient="records")
parsed_included_pbdb = json.loads(included_pbdb)

with open("../../dataset/pbdb/pbdb.json","w") as f:
    json.dump(parsed_included_pbdb,f)